In [16]:
import os
import io
import glob
import pprint


import pandas as pd

from bson.objectid import ObjectId
import pymongo

import climatools.lblnew.bestfit_params as bestfits
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.setup_overlap as setup_overlap
import climatools.lblnew.pipeline as lblnew_pipeline

import climatools.cliradlw.setup as setup_clirad
import climatools.cliradlw.pipeline as clirad_pipeline

In [2]:
client = pymongo.MongoClient('localhost:27017')

In [3]:
db = client['kdistnew']

bestfit = db['bestfit']
overlap = db['overlap']
clirad = db['clirad']

In [84]:
def bestfit_outputfiles():
    return ['kg_nonlin.dat',
            'kg_lin.dat',
            'output_wcoolrg.dat',
            'output_wfluxg.dat',
            'output_flux.dat',
            'output_fluxg.dat',
            'output_coolr.dat',
            'output_coolrg.dat',
            'abscom.dat',
            'dgdgs.dat']


def insert_all_bestfits():
    for molecule, band in setup_bestfit.implemented_moleculebands():
        param = bestfits.kdist_params(molecule=molecule, band=band)
        dir_fortran = lblnew_pipeline.get_dir_case(param, 
                                                   setup=setup_bestfit)
        
        c_output = {}
        for name in bestfit_outputfiles():
            with open(os.path.join(dir_fortran, name), 
                      mode='r', encoding='utf-8') as f:
                c = f.read()      
        c_output[name.split('.')[0]] = c
        
        case = {'input': param, 'output': c_output}    
        bestfit.insert_one(case)

In [6]:
def insert_all_cliradlw():
    molecule = 'h2o'
    band = '3'
    
def param_cliradlw(molecule):
    pass
    

In [23]:
for n in []:
    print(n)

In [21]:
def get_run_directories():
    rootdir_runs = os.path.join('/chia_cluster/home/jackyu',
                                'radiation/crd/LW/examples',
                                'separate_g_groups',
                                'study__lblnew_g1_threshold')
    for cdir, dirnames, fnames in os.walk(rootdir_runs):
        if dirnames:
            continue
        else:
            d = {}
            
            param = path2param(cdir)
            
            for name in fnames:
                if name.endswith('.f'):
                    
                    c = readtext(name)
                    
                    
                elif name.endswith('.dat'):
                    c = readtext(name)
                
                

In [22]:
get_run_directories()

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_0_o3_0_n2o_1_ch4_1_o2_0/band07_wn_1215_1380/nv_200/dv_0.005/crd_443e4ac/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_0_o3_0_n2o_1_ch4_1_o2_0/band07_wn_1215_1380/nv_200/dv_0.005/crd_443e4ac/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_0_o3_0_n2o_1_ch4_1_o2_0/band07_wn_1215_1380/nv_200/dv_0.005/crd_443e4ac/atmpro_trp
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_0_o3_0_n2o_1_ch4_1_o2_0/band07_wn_1215_1380/nv_200/dv_0.005/crd_736df33/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_0_o3_0_n2o_1_ch4_1_o2_0/band07_wn_1215_1380/nv_200/dv_0.005/crd_736df33/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/example

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band08_wn_1380_1900/nv_20/dv_0.05/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/wgt_0.55_0.55_0.85__0.85_0.85__0.0_0.3_0.55/wgt_flux_2/wgt_k_1/klin_none/w_diffuse_1.66_1.66_1.66__1.66_1.66__1.66_1.66_1.66/crd_90e01dd/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band08_wn_1380_1900/nv_1000/dv_0.001/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/ng_adju_0__-1/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.55_0.55_0.85__0.85_0.85__0.0_0.3_0.55/wgt_flux_2/w_diffuse_1.66_1.66_1.66__1.66_1.66__1.66_1.66_1.66/option_compute_ktable_0/option_compute_btable_0/crd_206550a
/chia_cluster/home/jackyu/radiation/crd/L

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band01_wn_20_340/nv_50/dv_0.02/ng_10/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__2__5/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/wgt_0.3_0.3_0.3__0.3_0.3__0.3_0.6_0.6_0.6_0.6/wgt_flux_2/wgt_k_1/klin_none/w_diffuse_1.6_1.8_1.8__1.8_1.7__1.6_1.4_1.4_1.7_1.8/crd_tmp/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band01_wn_20_340/nv_50/dv_0.02/ng_10/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__2__5/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/wgt_0.3_0.3_0.3__0.3_0.3__0.3_0.6_0.6_0.6_0.6/wgt_flux_2/wgt_k_1/klin_none/w_diffuse_1.6_1.8_1.8__1.8_1.7__1.6_1.4_1.4_1.7_1.8/crd_435ed9a/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band01_wn

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/conc_None/band08_wn_1380_1900/nv_1000/dv_0.001/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/ng_adju_0__-1__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_saw/wgt_k_1/wgt_0.55_0.55_0.85__0.85_0.85__0_0.3_0.55/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66__1.66_1.66_1.66/option_compute_ktable_0/option_compute_btable_0/crd_d62b618
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/conc_None/band08_wn_1380_1900/nv_1000/dv_0.001/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/ng_adju_0__-1__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_saw/wgt_k_1/wgt_0.55_0.55_0.85__0.85_0.85__0_0.3_0.55/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66__1.66_1.66_1.66/option_compute

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/conc_None/band07_wn_1215_1380/nv_1000/dv_0.001/ng_7/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_7/ng_adju_0/getabsth_auto/absth_dlogN_uniform/klin_none/atmpro_saw/wgt_k_1/wgt_0.5_0.5_0.5_0.5_0.5_0.5_0.9/wgt_flux_1/w_diffuse_2_1.6_1.6_1.6_1.6_1.6_1.8/option_compute_ktable_0/option_compute_btable_0/crd_5014a19
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/conc_None/band07_wn_1215_1380/nv_1000/dv_0.001/ng_7/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_7/ng_adju_0/getabsth_auto/absth_dlogN_uniform/klin_none/atmpro_trp/wgt_k_1/wgt_0.5_0.5_0.5_0.5_0.5_0.5_0.9/wgt_flux_1/w_diffuse_2_1.6_1.6_1.6_1.6_1.6_1.8/option_compute_ktable_0/option_compute_btable_0/crd_d62b618
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/conc_None/band07_wn_1215_1380/nv_1000/dv_0.001/ng_7/g_ascendi

...

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_1_o3_1_n2o_0_ch4_0_o2_0/band05_wn_980_1100/nv_1000/dv_0.001/crd_736df33/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_1_o3_1_n2o_0_ch4_0_o2_0/band05_wn_980_1100/nv_1000/dv_0.001/crd_736df33/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_1_o3_1_n2o_0_ch4_0_o2_0/band05_wn_980_1100/nv_1000/dv_0.001/crd_736df33/atmpro_trp
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_1_o3_1_n2o_0_ch4_0_o2_0/band05_wn_980_1100/nv_1000/dv_0.001/crd_5014a19/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o_1_co2_1_o3_1_n2o_0_ch4_0_o2_0/band05_wn_980_1100/nv_1000/dv_0.001/crd_5014a19/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/example

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/o3/band05_wn_980_1100/nv_200/dv_0.005/ng_7/g_ascending_k_descending/refPTs_P_1_T_250__P_50_T_250/ng_refs_2__5/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/wgt_0.35_0.6__0.5_0.55_0.7_0.9_1.0/wgt_flux_2/wgt_k_1/klin_2e-20/w_diffuse_1.6_1.75__1.55_1.66_1.7_1.75_1.8/crd_9d81085/atmpro_mls
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/o3/band05_wn_980_1100/nv_200/dv_0.005/ng_7/g_ascending_k_descending/refPTs_P_1_T_250__P_50_T_250/ng_refs_2__5/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/wgt_0.35_0.6__0.5_0.55_0.7_0.9_1.0/wgt_flux_2/wgt_k_1/klin_2e-20/w_diffuse_1.6_1.75__1.55_1.66_1.7_1.75_1.8/crd_9d81085/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/o3/band05_wn_980_1100/nv_200/dv_0.005/ng_7/g_ascending_k_descending/refPTs_P_1_T_250__P_50_T_250/ng_refs_2__5/getabst

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/band07_wn_1215_1380/P_50_T_250/min_4e-21_max_1e-17_ng_6_dlogN_.3_.3_.1_.1_.1/wgt_1_.8_.7_.6_.6_.6_.6/midlatitude_summer
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/band07_wn_1215_1380/P_50_T_250/min_4e-21_max_1e-17_ng_4/wgt_1_.8_.7_.6/midlatitude_summer
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/band07_wn_1215_1380/P_50_T_250/min_4e-21_max_1e-17_ng_4/wgt_1_.8_.7_.6/subarctic_winter
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/band07_wn_1215_1380/P_500_T_250/tmp_sum_xtemp
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/band07_wn_1215_1380/P_500_T_250/5.60488E-20_r8
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/ban

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/conc_3.2e-07/band07_wn_1215_1380/nv_1000/dv_0.001/ng_4/g_ascending_k_descending/refPTs_P_1_T_250__P_500_T_250/ng_refs_2__2/ng_adju_0__0/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/klin_2.22e-20/atmpro_mls/wgt_k_1/wgt_0.6_0.6__0.7_0.9/wgt_flux_1/w_diffuse_1.8_1.66__1.5_1.8/option_compute_ktable_1/option_compute_btable_0/crd_d5931a1
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/conc_3.2e-07/band07_wn_1215_1380/nv_1000/dv_0.001/ng_4/g_ascending_k_descending/refPTs_P_1_T_250__P_500_T_250/ng_refs_2__2/ng_adju_0__0/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/klin_2.22e-20/atmpro_mls/wgt_k_1/wgt_0.6_0.6__0.7_0.9/wgt_flux_2/w_diffuse_1.8_1.66_1.5_1.8/option_compute_ktable_0/option_compute_btable_0/crd_f17e0b2
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/conc_3.2e-

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/conc_3.2e-07/band03a_wn_540_620/nv_1000/dv_0.001/ng_3/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_1__1__1/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_1.94e-19/atmpro_saw/wgt_k_1/wgt_0.9__0.9__0.9/wgt_flux_2/w_diffuse_1.66__1.66__1.66/option_compute_ktable_0/option_compute_btable_0/crd_f17e0b2
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/n2o/conc_3.2e-07/band03a_wn_540_620/nv_1000/dv_0.001/ng_3/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_1__1__1/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_1.94e-19/atmpro_trp/wgt_k_1/wgt_0.8__0.8__0.8/wgt_flux_2/w_diffuse_1.66__1.66__1.66/option_compute_ktable_0/option_compute_btable_0/crd_f17e0b2
/chia_cluster/home/jackyu/radiation/crd/LW

In [100]:
bestfit.delete_one({'_id': ObjectId('5b28a260af10727d0e6a907c')})

In [4]:
for r in bestfit.find({'input.band': '3b'},
                      projection={'input.molecule': 1,
                                  'input.band': 1,
                                  'input.atmpro': 1,
                                  'input.commitnumber': 1,
                                  'output.dgdgs': 1}):
    print(r['_id'])
    pprint.pprint(r['input'])
    print(r['output']['dgdgs'])

5b28b908af10727d0e6a9081
{'atmpro': 'trp', 'band': '3b', 'commitnumber': 'a06b618', 'molecule': 'h2o'}
       g                  dg                 dgs
       1      0.74500000E-02      0.76657311E-02
       2      0.38520000E-01      0.38841090E-01
       3      0.92910000E-01      0.93707735E-01
       4      0.24753000E+00      0.24927679E+00
       5      0.38811000E+00      0.39011124E+00
       6      0.22548000E+00      0.22039741E+00

5b28b909af10727d0e6a908a
{'atmpro': 'mls', 'band': '3b', 'commitnumber': 'a06b618', 'molecule': 'co2'}
       g                  dg                 dgs
       1      0.12800000E-02      0.12842906E-02
       2      0.42500000E-02      0.42677704E-02
       3      0.19960000E-01      0.20046012E-01
       4      0.65950000E-01      0.66214986E-01
       5      0.24303000E+00      0.24386821E+00
       6      0.47273000E+00      0.47310358E+00
       7      0.19280000E+00      0.19121514E+00

